In [4]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguedashplayerstats  # import the new endpoint
import pandas as pd

#Set Seasons wanted here
start_season = 2014 #This will start with 2014_15
end_season = 2022  #This will end with 2022_23

basic_stats = pd.DataFrame()  # rename the dataframe to basic_stats

# loop through each season from 2014/15 to 2022/23
for season in range(start_season, end_season):

    # set up parameters for API request
    year = str(season) + "-" + str(season + 1)[-2:]
    season_type = "Regular Season"
    per_mode = "PerGame"

    # make API request with the new endpoint
    response = leaguedashplayerstats.LeagueDashPlayerStats(
        season=year,
        season_type_all_star=season_type,
        per_mode_detailed=per_mode,
    )

    # get dataframe from response and append to overall dataframe
    df = response.get_data_frames()[0]
    df['SEASON'] = year  # add new column for season
    basic_stats = basic_stats.append(df)

# reset index of dataframe
basic_stats = basic_stats.reset_index(drop=True)

basic_stats.head()


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SEASON
0,201985,AJ Price,AJ,1610612739,CLE,28.0,26,11,15,0.423,...,369,470,261,321,384,371,252,21,365,2014-15
1,201166,Aaron Brooks,Aaron,1610612741,CHI,30.0,82,50,32,0.610,...,93,109,170,124,70,179,193,21,159,2014-15
2,203932,Aaron Gordon,Aaron,1610612753,ORL,19.0,47,14,33,0.298,...,269,245,289,317,428,306,193,21,311,2014-15
3,203940,Adreian Payne,Adreian,1610612750,MIN,24.0,32,7,25,0.219,...,272,37,236,254,486,245,122,21,245,2014-15
4,201143,Al Horford,Al,1610612737,ATL,29.0,76,56,20,0.737,...,162,282,188,64,35,36,46,13,49,2014-15


In [5]:
basic_stats.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK',
       'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK',
       'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK',
       'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK',
       'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
       'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK',
       'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK', 'SEASON'],
      dtype='object')

In [6]:
player_team = basic_stats[['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ABBREVIATION', 'SEASON', 'AGE', 'FG_PCT', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PLUS_MINUS']]
player_team.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,AGE,FG_PCT,PTS,REB,AST,STL,BLK,TOV,PLUS_MINUS
0,201985,AJ Price,CLE,2014-15,28.0,0.372,5.1,1.2,1.8,0.3,0.0,0.5,-2.5
1,201166,Aaron Brooks,CHI,2014-15,30.0,0.421,11.6,2.0,3.2,0.7,0.2,1.9,2.5
2,203932,Aaron Gordon,ORL,2014-15,19.0,0.447,5.2,3.6,0.7,0.4,0.5,0.8,-3.5
3,203940,Adreian Payne,MIN,2014-15,24.0,0.414,6.7,5.1,0.9,0.6,0.3,1.4,-7.6
4,201143,Al Horford,ATL,2014-15,29.0,0.538,15.2,7.2,3.2,0.9,1.3,1.3,3.8


In [7]:
# Rename columns
player_team = player_team.rename(columns={'PLAYER_ID': 'pid', 'PLAYER_NAME': 'name',
                                          'TEAM_ABBREVIATION': 'team', 'SEASON': 'season'})

# Create new column "id"
player_team['id'] = player_team['pid'].astype(str) + '_' + player_team['season']

# Preview the dataframe
player_team.head()

,pid,name,team,season,AGE,FG_PCT,PTS,REB,AST,STL,BLK,TOV,PLUS_MINUS,id
0,201985,AJ Price,CLE,2014-15,28.0,0.372,5.1,1.2,1.8,0.3,0.0,0.5,-2.5,201985_2014-15
1,201166,Aaron Brooks,CHI,2014-15,30.0,0.421,11.6,2.0,3.2,0.7,0.2,1.9,2.5,201166_2014-15
2,203932,Aaron Gordon,ORL,2014-15,19.0,0.447,5.2,3.6,0.7,0.4,0.5,0.8,-3.5,203932_2014-15
3,203940,Adreian Payne,MIN,2014-15,24.0,0.414,6.7,5.1,0.9,0.6,0.3,1.4,-7.6,203940_2014-15
4,201143,Al Horford,ATL,2014-15,29.0,0.538,15.2,7.2,3.2,0.9,1.3,1.3,3.8,201143_2014-15


In [8]:
import os
relative_path_to_csvs = "../output"
filename = 'masterPartition.csv'
fullpath = os.path.join(relative_path_to_csvs, filename)

players = pd.read_csv(fullpath)

In [9]:
players = players.drop('idx', axis=1)
players.head()

,id,group,pos,name,season,GroupYear
0,1495_2014-15,2,C,Tim Duncan,2014-15,Group 2 (2014-15)
1,2207_2014-15,1,SG,Joe Johnson,2014-15,Group 1 (2014-15)
2,202693_2014-15,2,PF,Markieff Morris,2014-15,Group 2 (2014-15)
3,1717_2014-15,2,PF,Dirk Nowitzki,2014-15,Group 2 (2014-15)
4,101108_2014-15,1,PG,Chris Paul,2014-15,Group 1 (2014-15)


In [10]:
# Merge dataframes with left join
pos_team = players.merge(player_team, on=['name', 'id', 'season'], how='left')

# Fill missing values with "No Data"
pos_team = pos_team.fillna("No Data")

pos_team.head()

,id,group,pos,name,season,GroupYear,pid,team,AGE,FG_PCT,PTS,REB,AST,STL,BLK,TOV,PLUS_MINUS
0,1495_2014-15,2,C,Tim Duncan,2014-15,Group 2 (2014-15),1495.0,SAS,39.0,0.512,13.9,9.1,3.0,0.8,2.0,1.7,4.0
1,2207_2014-15,1,SG,Joe Johnson,2014-15,Group 1 (2014-15),2207.0,BKN,34.0,0.435,14.4,4.8,3.7,0.7,0.2,1.7,-1.5
2,202693_2014-15,2,PF,Markieff Morris,2014-15,Group 2 (2014-15),202693.0,PHX,25.0,0.465,15.3,6.2,2.3,1.2,0.5,2.1,1.3
3,1717_2014-15,2,PF,Dirk Nowitzki,2014-15,Group 2 (2014-15),1717.0,DAL,37.0,0.459,17.3,5.9,1.9,0.5,0.4,1.1,3.1
4,101108_2014-15,1,PG,Chris Paul,2014-15,Group 1 (2014-15),101108.0,LAC,30.0,0.485,19.1,4.6,10.2,1.9,0.2,2.3,8.9


In [11]:
pos_team.to_csv('nodeData.csv', index=False)